In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyLDAvis.gensim
import re
import nltk
from nltk.stem.porter import *
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import gensim
from gensim import corpora, models
import pickle

%matplotlib inline


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
data = pd.read_pickle("../ETL/data.pkl")
review_pd = pd.read_pickle("../ETL/review_pd.pkl")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
data.head()


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,name,seller,price,original_price,lectures,hours,level,enrollment,rating,...,downloadable_resources,last_update_date,instructor_title,instructor_job_title,instructor_info,instructor_rating,instructor_no_reviews,instructor_no_students,instructor_no_courses,review_index
0,1,the-complete-sql-bootcamp/,Jose Portilla,25.99,179.99,83,9.0,All Levels,342288.0,4.7,...,14.0,2020-10-01,Jose Portilla,"Head of Data Science, Pierian Data Inc.",Jose Marcial Portilla has a BS and MS in Mec...,4.6,712729.0,2179902.0,31.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
1,2,tableau10/,Kirill Eremenko SuperDataScience Team,16.99,124.99,82,9.0,All Levels,231631.0,4.6,...,6.0,2021-02-01,Ligency Team,Helping Data Scientists Succeed,"Hi there, We are the Ligency PR and Marketing ...",4.5,429183.0,1472140.0,48.0,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]"
2,3,powerbi-complete-introduction/,Manuel Lorenz Academind by Maximilian Schwarz...,12.99,94.99,275,23.5,Beginner,148077.0,4.6,...,26.0,2020-11-01,Maximilian Schwarzmüller,Professional Web Developer and Instructor,Experience as (Web) Developer Starting out at ...,4.6,564184.0,1335299.0,35.0,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]"
3,4,microsoft-power-bi-up-running-with-power-bi-de...,Maven Analytics Chris Dutton Aaron Parry,19.99,139.99,117,11.0,All Levels,109303.0,4.6,...,14.0,2021-02-01,Aaron Parry,Professional Business Intelligence Trainer & C...,Aaron is a professional analytics consultant a...,4.6,34377.0,438163.0,3.0,"[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]"
4,5,the-business-intelligence-analyst-course-2018/,365 Careers,12.99,94.99,413,20.5,All Levels,136869.0,4.5,...,151.0,2020-11-01,365 Careers,Creating opportunities for Business & Finance ...,365 Careers is the #1 best-selling provider of...,4.5,385370.0,1322640.0,68.0,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]"


In [4]:
print(data.columns)


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['id', 'name', 'seller', 'price', 'original_price', 'lectures', 'hours',
       'level', 'enrollment', 'rating', 'num_ratings', 'five_stars',
       'four_stars', 'three_stars', 'two_stars', 'description',
       'hours_ondemand_video', 'downloadable_resources', 'last_update_date',
       'instructor_title', 'instructor_job_title', 'instructor_info',
       'instructor_rating', 'instructor_no_reviews', 'instructor_no_students',
       'instructor_no_courses', 'review_index'],
      dtype='object')


## LDA for course description

In [9]:
docs = data["description"].to_list()
print(len(docs), "course descriptions are collected.")

1009 course descriptions are collected.


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
docs[0]

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"Use SQL to query a database Use SQL to perform data analysis Be comfortable putting SQL and PostgreSQL on their resume Learn to perform GROUP BY statements Replicate real world situations and query reports Requirements No prior technical experience is required  All you need a computer  Description Learn how to use SQL quickly and effectively with this course! You'll learn how to read and write complex queries to a database using one of the most in demand skills - PostgreSQL. These skills are also applicable to any other major SQL database, such as MySQL, Microsoft SQL Server, Amazon Redshift, Oracle, and much more. Learning SQL is one of the fastest ways to improve your career prospects as it is one of the most in demand tech skills! In this course you'll learn quickly and receive challenges and tests along the way to improve your understanding! In this course you will learn everything you need to become a SQL Pro! Including: Get started with PostgreSQL and PgAdmin , two of the world'

In [14]:
def preprocess_sent(text):
    """
    Get sentence level preprocessed data from raw texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    # Add missing delimiter: xxxThis is a --> xxx.
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', text)  # before lower case
    # Make words all lower case.
    s = s.lower()
    # Remove < and >
    s = re.sub(r'&gt|&lt', ' ', s)
    # Remove letters that repeat more than 2 time.
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # Remove non-word characters that repeat more than 1 time.
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # Use string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # Remove words in parenthesis, which are assumed less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # [.?!]xxx --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    return s.strip()

text_lists = [preprocess_sent(text) for text in docs]

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: expected string or bytes-like object